<a href="https://colab.research.google.com/github/huskeypm/cellmigration/blob/main/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# master
Workhorse for generating and executing jobs in support of migration paper.

Once the csv files are made from the postprocessing, use the colab notebook to plot:


NEEDS TO RUN ON KANT FOR THE TIME BEING


In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
#!git clone https://github.com/huskeypm/cellmigration
!cd cellmigration; git pull

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 12 (delta 8), reused 12 (delta 8), pack-reused 0
Unpacking objects: 100% (12/12), 1.97 KiB | 336.00 KiB/s, done.
From https://github.com/huskeypm/cellmigration
   510b5a8..d257e4c  main       -> origin/main
Updating 510b5a8..d257e4c
Fast-forward
 batchProcess.py          |   61 +-
 brown_util.py            |   26 +-
 master.ipynb             | 4927 ------------------------------------------------------------------
 paper/template_full.yaml |   44 +-
 paper/writeInputs.py     |    5 +-
 5 files changed, 82 insertions(+), 4981 deletions(-)
 delete mode 100644 master.ipynb


In [23]:
import sys
sys.path.append("/content/cellmigration/")

In [24]:
ls /content/cellmigration/

batchProcess.py  brown_wnonbond.py          histo.ipynb            pdb.py          tests/
brownian_v2.py   calculator.py              lattice.py             processYaml.py  update.py
brownian_v3.py*  concentration.py           LICENSE                README_1.md
brownian_v4.py*  conda_openmm_pytraj.ipynb  migration_paper.ipynb  README.md
brownian_v5.py*  config.bash                paper/                 run/
brown_util.py    expt_analysis/             parameters.py*         states.py


In [31]:
import yaml
import sys

sys.path.append('/content/cellmigration/paper/')
import writeInputs as wi
#import batchProcess as bp
#outDir = "/home/pkh-lab-shared/migration/231110/"
outDir = "/home/pkekeneshuskey/data/231208/"
outDir = "./"
# FOR LOCAL
cmd = "python3 brownian_v3.py -yamlFile " # if local

# FOR FAUST/W
cmd='$EXEC ' # if remote host

wi.params = wi.Params(
    cmd=cmd,
    path=outDir
)
#wi.params.cmd = cmd
#wi.params.path=outDir

print("ADJ PATH LATER")
#bp.path=outDir
print(wi.params.cmd)

varIter = 7 # keys
varRuns = 5 # replicates per run

ADJ PATH LATER
$EXEC 


The idea of the reference file is to have a complete system that toggled - for instance, we have a case that is tuned for ATP application, but we can set the ATP concentration to zero to emulate a ATP-free condition. Also want values that are intermediate, so when we vary them by 50%-200%, we get reasonable numbers

In [26]:
referenceFile="cellmigration/paper/template_full.yaml"
with open(referenceFile, 'r') as file:
     daVars = yaml.safe_load(file)

## crowder free case
### No ATP
Can copy/paste into file to run


In [36]:
nocrowders = daVars.copy()
nocrowders['nCrowders']=1
nocrowders['crowderRad']=0.1
nocrowders['outName']='nocrowder'
keys=['nCells','cellRad','cellAttr']

In [37]:
nocrowders

{'nCells': 60,
 'cellAttr': 0.1,
 'cellRad': 4,
 'friction': 25,
 'crowderAttr': 0.1,
 'nCrowders': 1,
 'crowderRad': 0.1,
 'states': False,
 'tag': 'nCells',
 'xPotential': False,
 'xScale': 5.0,
 'yPotential': False,
 'yScale': 20,
 'containmentPotential': 'square',
 'crowderXDim': 50,
 'crowderYDim': 100,
 'absorbingBoundary': True,
 'absorbingMargin': 10.0,
 'domainXDim': 175,
 'domainYDim': 100,
 'nUpdates': 2500000,
 'outName': 'nocrowder'}

In [38]:

####### cellAttr ----> particleAttr
for daKey in keys:
    print("Generate ",daKey)
    writeFile = nocrowders['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowders,daKey,varIter=varIter, varRuns=varRuns,
                         writeFile=writeFile)


Generate  nCells
nCells [ 30.          37.7976315   47.62203156  60.          75.59526299
  95.24406312 120.        ]
Wrote nocrowder_nCells.bash
Generate  cellRad
cellRad [2.         2.5198421  3.1748021  4.         5.0396842  6.34960421
 8.        ]
Wrote nocrowder_cellRad.bash
Generate  cellAttr
cellAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote nocrowder_cellAttr.bash


In [ ]:
bp.processYamls(nocrowders['outName'],yamlNamePrefix=nocrowders['outName'],prefixOptions=keys,
               display=False)

### + ATP



In [45]:
nocrowdersatp = daVars.copy()
nocrowdersatp['nCrowders']=1
nocrowdersatp['crowderRad']=0.1
nocrowdersatp['outName']='nocrowder_atp'
nocrowdersatp['xPotential']=True
keys=['nCells','cellRad','cellAttr','xScale','yScale']
print(wi.params.path)

./


In [46]:


for daKey in keys:
    print("Generate ",daKey)
    writeFile = nocrowdersatp['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowdersatp,daKey,varIter=varIter, varRuns=varRuns,
                         writeFile=writeFile)



Generate  nCells
nCells [ 30.          37.7976315   47.62203156  60.          75.59526299
  95.24406312 120.        ]
Wrote nocrowder_atp_nCells.bash
Generate  cellRad
cellRad [2.         2.5198421  3.1748021  4.         5.0396842  6.34960421
 8.        ]
Wrote nocrowder_atp_cellRad.bash
Generate  cellAttr
cellAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote nocrowder_atp_cellAttr.bash
Generate  xScale
xScale [ 2.5         3.14980262  3.96850263  5.          6.29960525  7.93700526
 10.        ]
Wrote nocrowder_atp_xScale.bash
Generate  yScale
yScale [10.         12.5992105  15.87401052 20.         25.198421   31.74802104
 40.        ]
Wrote nocrowder_atp_yScale.bash


In [ ]:
bp.processYamls(nocrowdersatp['outName'],
                yamlNamePrefix=nocrowdersatp['outName'],prefixOptions=keys,
                display=False
               )

NameError: name 'nocrowdersatp' is not defined

## crowder  case
### No ATP

In [47]:
crowders = daVars.copy()
crowders['outName']='crowder'
keys=['nCrowders','crowderRad','crowderAttr']
print(wi.params.path)

./


In [48]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowders['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    wi.WriteIterativeYaml(crowders,daKey,varIter=varIter, varRuns=varRuns,
                         writeFile=writeFile)

Generate  nCrowders
nCrowders [ 9.         11.33928945 14.28660947 18.         22.6785789  28.57321894
 36.        ]
Wrote crowder_nCrowders.bash
Generate  crowderRad
crowderRad [ 5.          6.29960525  7.93700526 10.         12.5992105  15.87401052
 20.        ]
Wrote crowder_crowderRad.bash
Generate  crowderAttr
crowderAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote crowder_crowderAttr.bash


Loaded /home/pkh-lab-shared/migration/231110/crowder_nCrowders9.000000_00.dcd


In [ ]:
bp.processYamls(crowders['outName'],yamlNamePrefix=crowders['outName'],prefixOptions=keys)

### +ATP

In [50]:
crowdersatp = daVars.copy()
crowdersatp['outName']='crowder_atp'
crowdersatp['xPotential']=True
keys=['nCrowders','crowderRad','crowderAttr']
print(wi.params.path)

./


In [51]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowdersatp['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    wi.WriteIterativeYaml(crowdersatp,daKey,varIter=varIter, varRuns=varRuns,
                         writeFile=writeFile)

Generate  nCrowders
nCrowders [ 9.         11.33928945 14.28660947 18.         22.6785789  28.57321894
 36.        ]
Wrote crowder_atp_nCrowders.bash
Generate  crowderRad
crowderRad [ 5.          6.29960525  7.93700526 10.         12.5992105  15.87401052
 20.        ]
Wrote crowder_atp_crowderRad.bash
Generate  crowderAttr
crowderAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote crowder_atp_crowderAttr.bash


In [ ]:
bp.processYamls(crowdersatp['outName'],yamlNamePrefix=crowdersatp['outName'],prefixOptions=keys)

/home/pkh-lab-shared/migration/231110/crowder_atp_nCrowders4.500000_00.yaml
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_00
nCrowders 4.5
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_00.dcd
Di 0.021116 J -0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_01
nCrowders 4.5
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_01.dcd
Di 0.016905 J 0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_02
nCrowders 4.5
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_02.dcd
Di 0.013919 J -0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_03
nCrowders 4.5
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_03.dcd


./paper/batchProcess.py:65: RuntimeWarning: divide by zero encountered in log


Di 0.017418 J 0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_04
nCrowders 4.5
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders4.500000_04.dcd
Di 0.019076 J -0.003003
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_00
nCrowders 5.669644724526929
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_00.dcd
Di 0.017065 J 0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_01
nCrowders 5.669644724526929
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_01.dcd
Di 0.015231 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_02
nCrowders 5.669644724526929
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_02.dcd
Di 0.017023 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_nCrowders5.669645_03
nCrowders 5.669644724526929
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_nCrowder

/home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_01
crowderRad 11.905507889761495
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_01.dcd
Di 0.013515 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_02
crowderRad 11.905507889761495
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_02.dcd
Di 0.018482 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_03
crowderRad 11.905507889761495
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_03.dcd
Di 0.014122 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_04
crowderRad 11.905507889761495
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad11.905508_04.dcd
Di 0.023730 J -0.002002
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderRad15.000000_00
crowderRad 15.0
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_cro

/home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.000000_03
crowderAttr 1.0
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.000000_03.dcd
Di 0.016157 J -0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.000000_04
crowderAttr 1.0
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.000000_04.dcd
Di 0.019997 J -0.001001
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.259921_00
crowderAttr 1.2599210498948732
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.259921_00.dcd
Di 0.010710 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.259921_01
crowderAttr 1.2599210498948732
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.259921_01.dcd
Di 0.017527 J 0.000000
/home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.259921_02
crowderAttr 1.2599210498948732
Loaded /home/pkh-lab-shared/migration/231110//crowder_atp_crowderAttr1.

## Other testing